In [3]:
import re
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from rapidfuzz import fuzz
import numpy as np

sys.path.append(os.path.abspath(".."))

from utils.db import fetch_sanctions

In [ ]:
df = fetch_sanctions()
df.head()

In [ ]:
# Data Overview

df.info()
df.describe(include="all")
df.isnull().sum()

In [ ]:
# Sanity check for uniqueness
df['ent_num'].nunique(), df.duplicated().sum()

In [ ]:
# Data Distribution (Exploratory Visuals)

sns.countplot(data=df, y='country', order=df['country'].value_counts().head(10).index)
plt.title("Top 10 Countries in Sactions List")
plt.show()

In [ ]:
df['sdn_type'].value_counts().plot(kind='barh', title="SDN Types Distribution")

In [ ]:
# Data Cleaning

def standardize_name(name: str) -> str:
    if pd.isnull(name):
        return ''
    name = re.sub(r"[/-]", " ", name).upper()
    name = re.sub(r"[^A-Z\s]", "", name)
    name = re.sub(r"\s+", " ", name).strip()
    return name

df['cleaned_name'] = df['sdn_name'].apply(standardize_name)

df.head(10)

In [ ]:
# Feature Engineering

def get_fuzz_ratio(name1: str, name2: str) -> float:
    name1 = " ".join(sorted(name1.split()))
    name2 = " ".join(sorted(name2.split()))

    try:
        return round(fuzz.ratio(name1, name2), 2)
    except:
        return 0

def get_length_difference(name1: str, name2: str) -> int:
    return abs(len(name1) - len(name2))
    
df['name_length'] = df['cleaned_name'].str.len()
df['word_count'] = df['cleaned_name'].str.split().apply(len)
df['has_country_in_name'] = df.apply(
    lambda row: 1 if row['country'] and row['country'].upper() in row['cleaned_name'] else 0,
    axis=1
)

# Text similarity features
reference_name = standardize_name("AEROCARIBBEAN AIRLINES")
df['fuzz_ratio_reference'] = reference_name
df['fuzz_ratio'] = df['cleaned_name'].apply(lambda x: get_fuzz_ratio(x, reference_name))
df['length_diff'] = df['cleaned_name'].apply(lambda x: get_length_difference(x, reference_name))

df.head(10)


,ent_num,sdn_name,sdn_type,program,title,call_sign,vess_type,tonnage,grt,vess_flag,...,alt_type,alt_name,alt_remarks,cleaned_name,name_length,word_count,has_country_in_name,fuzz_ratio,fuzz_ratio_reference,length_diff
0,36,AEROCARIBBEAN AIRLINES,None,CUBA,None,None,None,None,None,None,...,aka,AERO-CARIBBEAN,None,AEROCARIBBEAN AIRLINES,22,2,0,100.00,AEROCARIBBEAN AIRLINES,0
1,173,"ANGLO-CARIBBEAN CO., LTD.",None,CUBA,None,None,None,None,None,None,...,aka,AVIA IMPORT,None,ANGLO CARIBBEAN CO LTD,22,4,0,59.09,AEROCARIBBEAN AIRLINES,0
2,306,BANCO NACIONAL DE CUBA,None,CUBA,None,None,None,None,None,None,...,aka,NATIONAL BANK OF CUBA,None,BANCO NACIONAL DE CUBA,22,4,0,40.91,AEROCARIBBEAN AIRLINES,0
3,306,BANCO NACIONAL DE CUBA,None,CUBA,None,None,None,None,None,None,...,aka,NATIONAL BANK OF CUBA,None,BANCO NACIONAL DE CUBA,22,4,0,40.91,AEROCARIBBEAN AIRLINES,0
4,306,BANCO NACIONAL DE CUBA,None,CUBA,None,None,None,None,None,None,...,aka,NATIONAL BANK OF CUBA,None,BANCO NACIONAL DE CUBA,22,4,0,40.91,AEROCARIBBEAN AIRLINES,0
5,306,BANCO NACIONAL DE CUBA,None,CUBA,None,None,None,None,None,None,...,aka,NATIONAL BANK OF CUBA,None,BANCO NACIONAL DE CUBA,22,4,0,40.91,AEROCARIBBEAN AIRLINES,0
6,424,BOUTIQUE LA MAISON,None,CUBA,None,None,None,None,None,None,...,None,None,None,BOUTIQUE LA MAISON,18,3,0,40.00,AEROCARIBBEAN AIRLINES,4
7,475,CASA DE CUBA,None,CUBA,None,None,None,None,None,None,...,None,None,None,CASA DE CUBA,12,3,0,35.29,AEROCARIBBEAN AIRLINES,10
8,475,CASA DE CUBA,None,CUBA,None,None,None,None,None,None,...,None,None,None,CASA DE CUBA,12,3,0,35.29,AEROCARIBBEAN AIRLINES,10
9,480,"CECOEX, S.A.",None,CUBA,None,None,None,None,None,None,...,None,None,None,CECOEX SA,9,2,0,32.26,AEROCARIBBEAN AIRLINES,13


In [23]:
df.to_csv('../data/sanctions_cleaned.csv', index=False)